In [1]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamplayerdashboard, playercareerstats
import json

import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from pickle import load

fileName = "history_dataset.csv"
modelName = "modelNBATrained_50.h5"
random_state = 42

In [2]:
teams_query = ['Los Angeles Lakers', 'Chicago Bulls']
current_season = '2023-24'
selected_columns = ['GP', 'GS', 'MIN', 'FG_PCT', 'FG3_PCT', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']
num_stats = len(selected_columns)

home_team = teams_query[0]
away_team = teams_query[1]
teams_query= [home_team, away_team]
season = current_season

array_1 = np.array([0.0] * num_stats)
array_2 = np.array([0.0] * num_stats)
teams_stats = [array_1, array_2]

for idx, team in enumerate(teams_query):
    t1 = teams.find_teams_by_full_name(team)
    team_id = t1[0]['id']
    print(team_id)

    t = teamplayerdashboard.TeamPlayerDashboard(team_id=team_id, season=season)
    data_player = json.loads(t.get_json())['resultSets']

    players_data = next(item for item in data_player if item['name'] == 'PlayersSeasonTotals')['rowSet']
    player_ids = [player[1] for player in players_data]
    print(f"{player_ids}\n")

    for player_id in player_ids:
        career = playercareerstats.PlayerCareerStats(player_id=player_id)
        df_single_player_carrer = career.get_data_frames()[0]
        last_row_stats = df_single_player_carrer[selected_columns].iloc[-1].tolist()
        #print(f"{last_row_stats}")
        teams_stats[idx] += np.array(last_row_stats)

pred_df = pd.DataFrame()
# Aggiungere le statistiche al DataFrame
for j, stat in enumerate(selected_columns):
    pred_df.loc[0, f'Home_Stat_{stat}'] = teams_stats[0][j]
    pred_df.loc[0, f'Away_Stat_{stat}'] = teams_stats[1][j]


1610612747
[1641788, 203076, 1630559, 1629629, 1626174, 1630658, 1626156, 1641793, 1629685, 1629216, 1628385, 1641720, 1629020, 1629637, 2544, 1631108, 1641721, 1629060, 1630219, 203915, 1627752]

1610612741
[1641766, 1627936, 203083, 1641847, 1630245, 1629632, 1631207, 201942, 1630929, 1629750, 1628975, 1641763, 202696, 1641931, 1630172, 1630678, 1628470, 203897]



,Home_Stat_GP,Away_Stat_GP,Home_Stat_GS,Away_Stat_GS,Home_Stat_MIN,Away_Stat_MIN,Home_Stat_FG_PCT,Away_Stat_FG_PCT,Home_Stat_FG3_PCT,Away_Stat_FG3_PCT,...,Home_Stat_STL,Away_Stat_STL,Home_Stat_BLK,Away_Stat_BLK,Home_Stat_TOV,Away_Stat_TOV,Home_Stat_PF,Away_Stat_PF,Home_Stat_PTS,Away_Stat_PTS
0,964.0,833.0,463.0,410.0,21871.0,19981.0,9.299,7.907,5.705,5.027,...,663.0,638.0,468.0,394.0,1189.0,957.0,1409.0,1541.0,10512.0,9206.0


In [3]:
with open('standardScaler_ready.pkl','rb') as f:
    scaler = load(f)
    
X_std = scaler.transform(pred_df)
X_std

array([[ 0.25186154, -0.73518555,  0.85616652,  0.16091684,  0.51792861,
        -0.09900181, -0.21698811, -1.18077905, -0.69766453, -1.3106905 ,
        -0.10022987, -1.15889776, -0.86982207,  0.23552377,  0.61213938,
        -0.1845492 ,  0.22135555, -0.07423736,  1.00801439, -0.44304672,
         0.31411035,  0.06705413,  0.53896805, -0.17793373,  0.64593153,
        -0.60665253, -0.74232348, -0.19808104,  0.71776772, -0.12804863]])

In [4]:
model = load_model(modelName)

In [5]:
res_tensor = model(X_std)
res_int = round(res_tensor.numpy().item())
res_int
if res_int == 1:
    print(f"Ha vinto la squadra: {teams_query[0]}")
else:
    print(f"Ha vinto la squadra: {teams_query[1]}")

Ha vinto squadra Chicago Bulls
